# --------------Chiến lược giao dịch theo chỉ số biến động------------------------------------------

# 1. Mã và thời gian: Du lieu cua SPY (S & P 500)

# Buoc 1: Load du lieu

In [16]:
import pandas as pd
import yfinance as yf

data = {}  # Tạo một từ điển để lưu trữ dữ liệu cho mỗi ký hiệu
def load_data(symbols, start_date, end_date):
    for symbol in symbols:
        data[symbol] = yf.download(symbol, start=start_date, end=end_date, interval='1d')
        data[symbol]['symbol'] = symbol
        data[symbol] = data[symbol].reset_index().rename(columns={'Adj Close': 'value', 'Date': 'date'}).sort_values(by = ['symbol', 'date'])
        data[symbol] = data[symbol][['symbol', 'date', 'value']]

    # Kết hợp dữ liệu từ các ký hiệu thành một DataFrame lớn
    dataAll = pd.concat(data.values(), ignore_index=True)

    # Pivot table dữ liệu lại
    dataAll = pd.pivot_table(dataAll, columns = 'symbol', index = 'date', values ='value')
    return dataAll

# Buoc 2: Viet ham kiem tra dieu kien mua hay ban

In [17]:
import pandas as pd

def decide_trade(data, vix_threshold = 30):
    """
    Quyết định mua hoặc bán SPY dựa trên giá trị VIX.

    :param dataframe: DataFrame chứa giá trị cho SPY, ^GSPC, và ^VIX.
    :param vix_threshold: Ngưỡng VIX để xác định biến động cao của thị trường.
    :return: Quyết định giao dịch: 'Mua', 'Bán', hoặc 'Giữ'.
    """
    latest_data = data.iloc[-1]  # Lấy dòng dữ liệu mới nhất: Index tu trai sang phai 0 -> len -1; tu ben phai qua -1 =>
    current_vix = latest_data['^VIX']

    if current_vix > vix_threshold:
        return 'Bán'  # Bán nếu VIX cao
    else:
        return 'Mua'  # Mua nếu VIX thấp

# Chay voi 1 phut 1 lan

In [18]:
import schedule
import time
from datetime import datetime, timedelta

symbols = ['SPY', '^GSPC', '^VIX']  # Danh sách các ký hiệu bạn muốn lấy dữ liệu

def execute_trade():
    end_date = datetime.now() # Lấy ngày hiện tại
    start_date = end_date - timedelta(days=1)  # Lấy ngày trước ngày hiện tại

    # Định dạng ngày tháng theo chuẩn (ví dụ: "2023-11-16")
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    data = load_data(symbols, start_date_str, end_date_str)
    ketqua = decide_trade(data)
    if ketqua == 'Mua':
        print("Thực hiện mua ở đây")
        # API mua: Mua bang API
    elif ketqua == 'Bán':
        print("Thực hiện bán ở đây")
        # API ban: Ban bang API

# Lên lịch hàm execute_trade để chạy mỗi phút
# schedule.every(1).minute.do(execute_trade)
schedule.every(1).day.at("21:44").do(execute_trade)

while True:
    schedule.run_pending()  # Kiểm tra và thực hiện các công việc lên lịch
    time.sleep(1)

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['SPY']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-15 -> 2024-01-16)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['^GSPC']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-15 -> 2024-01-16)')



[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['^VIX']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2024-01-15 -> 2024-01-16)')


IndexError: single positional indexer is out-of-bounds